# ETL-pipelines in Azure Databricks
## Module 3: Data Ingestie in Databricks
Deze notebook begeleidt je bij het laden van declaratiegegevens uit verschillende bronnen in Azure Databricks.

## Stap 1: Laad een CSV-bestand met declaraties

In [ ]:
from pyspark.sql import SparkSession

# Start een Spark sessie
spark = SparkSession.builder.appName("ETL_Declaraties").getOrCreate()

# Laad een CSV-bestand
df_csv = spark.read.csv("/mnt/data/declaraties_dataset.csv", header=True, inferSchema=True)

# Toon de eerste regels
df_csv.show(5)

## Stap 2: Laad een JSON-bestand met declaraties

In [ ]:
# Laad een JSON-bestand
df_json = spark.read.json("/mnt/data/declaraties.json")

# Toon de eerste regels
df_json.show(5)

## Stap 3: Laden vanuit een Azure SQL Database

In [ ]:
jdbc_url = "jdbc:sqlserver://menzis-db.database.windows.net:1433;database=DeclaratiesDB"
db_properties = {
    "user": "gebruikersnaam",
    "password": "wachtwoord",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Laad declaratiegegevens uit Azure SQL
df_sql = spark.read.jdbc(url=jdbc_url, table="Declaraties", properties=db_properties)

# Toon de eerste regels
df_sql.show(5)

## Stap 4: Laden vanuit Azure Data Lake Storage

In [ ]:
storage_account = "menzisdatalake"
container_name = "bronze"
file_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/declaraties_parquet"

# Laad een Parquet-bestand
df_parquet = spark.read.parquet(file_path)

# Toon de eerste regels
df_parquet.show(5)

## Stap 5: Data combineren en opslaan in Delta Lake

In [ ]:
from pyspark.sql.functions import col

# Zorg ervoor dat kolomnamen gelijk zijn
df_csv = df_csv.select("Verzekerde_ID", "Declaratie_ID", "Zorgtype", "Bedrag", "Declaratie_Datum", "Declaratie_Status")
df_json = df_json.select("Verzekerde_ID", "Declaratie_ID", "Zorgtype", "Bedrag", "Declaratie_Datum", "Declaratie_Status")

# Combineer CSV en JSON data
df_combined = df_csv.union(df_json)

# Toon het gecombineerde resultaat
df_combined.show(5)

In [ ]:
# Opslaan in Delta Lake
df_combined.write.format("delta").mode("overwrite").save("/mnt/data/declaraties_delta")

## Oefeningen
1. **Laad een JSON-bestand met declaraties in een DataFrame en inspecteer het schema.**
2. **Gebruik Spark SQL om declaraties per zorgtype te groeperen en het totaalbedrag te berekenen.**
3. **Filter de dataset zodat alleen declaraties uit de afgelopen 12 maanden overblijven.**
